In [1]:
import os
from datetime import datetime, date
from decimal import Decimal
from typing import List, Tuple

import pandas as pd
import matplotlib.pyplot as plt

from src.data_processing.fetch_data import fetch_oku_liquidity, fetch_pool_data
from src.models.pool import Pool
from src.models.liquidity_distribution import LiquidityDistribution
from src.utils.helper import organize_tick_data, sqrtPriceX96_to_price
from src.visualization.plot_liquidity import plot_liquidity_distribution
from config.api_config import API_KEY
from src.data_processing.swap_data_collector import SwapDataCollector

In [11]:
collector = SwapDataCollector(
    project_id="uniswap-v3-analytics",
    pool_address="0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",
    token0="WETH",
    token1="USDC",
    decimals0=6,
    decimals1=18,
    tick_spacing=60,
    start_date=date(2024, 8, 1),
    end_date=date(2024, 8, 31)
)

In [12]:
all_swaps = collector.load_swap_data()
processed_swaps = collector.process_swap_data(all_swaps)
sampled_blocks = collector.sample_blocks_dollar_bars(processed_swaps, 100000)

In [13]:
method = "dollar"
sampled_filename = os.path.join(collector.blocks_dir, f"{method}_bar_sampled_blocks.txt")
with open(sampled_filename, "w") as f:
    for block in sampled_blocks:
        f.write(f"{block}\n")

In [15]:
collector.load_sampled_blocks_bars(method='dollar')

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [8]:
import pandas as pd
from datetime import date, datetime
from decimal import Decimal
from typing import List, Tuple
import matplotlib.pyplot as plt
from src.models.pool import Pool
from src.models.liquidity_distribution import LiquidityDistribution
from src.data_processing.fetch_data import fetch_oku_liquidity
from src.utils.helper import organize_tick_data

class LiquidityAnalyzer:
    def __init__(self, pool_address: str, token0: str, token1: str, decimals0: int, decimals1: int, 
                 tick_spacing: int, start_date: date, end_date: date):
        self.pool_address = pool_address
        self.pool = Pool(token0, token1, decimals0, decimals1, tick_spacing)
        self.start_date = start_date
        self.end_date = end_date
        self.sampled_blocks = self.load_sampled_blocks()

    def load_sampled_blocks(self) -> pd.DataFrame:
        collector = SwapDataCollector(
            project_id="uniswap-v3-analytics",
            pool_address=self.pool_address,
            token0=self.pool.token0,
            token1=self.pool.token1,
            decimals0=self.pool.decimals0,
            decimals1=self.pool.decimals1,
            tick_spacing=self.pool.tick_spacing,
            start_date=self.start_date,
            end_date=self.end_date
        )
        blocks = collector.load_sampled_blocks_bars(method='dollar')
        
        df = pd.DataFrame(blocks)
        df['datetime'] = pd.to_datetime(df['block_timestamp'], unit='s')
        return df.sort_values('datetime')

    def select_blocks_by_date_range(self, start: date, end: date) -> List[int]:
        mask = (self.sampled_blocks['datetime'].dt.date >= start) & (self.sampled_blocks['datetime'].dt.date <= end)
        return self.sampled_blocks.loc[mask, 'block_number'].tolist()

    def analyze_liquidity(self, block_number: int) -> Tuple[List[dict], Decimal, Decimal]:
        data = fetch_oku_liquidity(pool_address=self.pool_address, block_number=block_number)
        tick_mapping = organize_tick_data(tick_data=data["ticks"])
        current_tick = int(data["current_pool_tick"])
        distribution = LiquidityDistribution(self.pool, tick_mapping, current_tick)
        result, total_amount0, total_amount1 = distribution.get_distribution()
        return result, total_amount0, total_amount1

    def plot_liquidity_over_time(self, start: date, end: date, num_samples: int = 5):
        blocks = self.select_blocks_by_date_range(start, end)
        sampled_blocks = blocks[:num_samples]  # Take first num_samples blocks for simplicity
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 16))
        
        for block in sampled_blocks:
            result, _, _ = self.analyze_liquidity(block)
            timestamp = self.sampled_blocks.loc[self.sampled_blocks['block_number'] == block, 'datetime'].iloc[0]
            
            ticks = [info.tick for info in result]
            liquidities = [info.liquidity for info in result]
            
            ax1.plot(ticks, liquidities, label=f'Block {block} ({timestamp:%Y-%m-%d %H:%M})')
            ax2.plot([info.price for info in result], liquidities, label=f'Block {block} ({timestamp:%Y-%m-%d %H:%M})')
        
        ax1.set_xlabel('Tick')
        ax1.set_ylabel('Liquidity')
        ax1.set_title('Liquidity Distribution over Ticks')
        ax1.legend()
        
        ax2.set_xlabel('Price')
        ax2.set_ylabel('Liquidity')
        ax2.set_title(f'Liquidity Distribution over Price ({self.pool.token1}/{self.pool.token0})')
        ax2.set_xscale('log')
        ax2.legend()
        
        plt.tight_layout()
        plt.show()

In [9]:
POOL_ADDRESS = "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"
START_DATE = date(2024, 8, 1)
END_DATE = date(2024, 8, 31)

analyzer = LiquidityAnalyzer(
    pool_address=POOL_ADDRESS,
    token0="WETH",
    token1="USDC",
    decimals0=18,
    decimals1=6,
    tick_spacing=60,
    start_date=START_DATE,
    end_date=END_DATE
)

ValueError: invalid literal for int() with base 10: "{'block_number': 20430121, 'block_timestamp': 1722472187}"

In [ ]:


# Analyze liquidity for a specific date range
analysis_start = date(2024, 8, 15)
analysis_end = date(2024, 8, 20)
analyzer.plot_liquidity_over_time(analysis_start, analysis_end)
